In [1]:
import tensorflow as tf
import numpy as np

## 특이사항

- learning_rate를 바꾸고 `saver.restore`를 하게 되면 `NotFoundError: Key Variable_3 not found in checkpoint`에러가 발생한다. 아마 모델을 변경하고 저장된 체크포인트를 가져오려고 하면 에러가 나는 듯함... 그럴땐 그냥 모델을 지워야함.

- 아무리 루프를 돌면서 학습해도 cost가 줄어들지 않는 경우가 있었다. -> W2, W3이 `tf.random_uniform`이 아닌 `tf.random_normal`이었다... `tf.random_uniform`으로 바꾸니까 잘되는듯.



In [2]:
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


- `unpack=True` parameter : 행과 열을 바꿔줌
- `np.transpose` : 행과 열을 바꿔줌

In [3]:
# training에 사용되는 변수가 아니고, 학습 회수를 카운트하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')


- `trainable=False` : 학습에 사용하지 않음 

In [11]:
# bias없이 가중치만 있는 모델

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.03)
train_op = optimizer.minimize(cost, global_step=global_step)



```
W1 : [2, 10] -> W2 : [10, 20] -> W3: [20, 3]
```
이런 식으로 신경망의 계층(W 추가)을 늘리고 은닉층 뉴런수(10, 20)를 증가시킬 수 있다. 그렇게 될 경우 복잡도가 높은 문제를 해결하는데 도움이 된다. 다만 이렇게 한다고 해서 무조건 도움이 되는 것은 아니며 과적합이라는 문제에 빠질 수도 있다.


In [12]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

- `tf.global_variables()` : 앞서 정의한 변수들을 가져오는 함수

In [13]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    try : 
        saver.restore(sess, ckpt.model_checkpoint_path)
    except tf.errors.NotFoundError as e:
        print("remove a model directory because of NotFoundError!")
        !rm -rf ./model
        sess.run(tf.global_variables_initializer())
    
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-100
remove a model directory because of NotFoundError!


위 코드는 `./model` 디렉토리에 기존에 학습해둔 모델이 있는지 확인하고 있다면 불러오고 없으면 변수를 새로 초기화 한다.

In [14]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if step % 10 == 0:
        print('Step : %d' % sess.run(global_step), 
             'Cost : %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step : 1 Cost : 0.664
Step : 11 Cost : 0.550
Step : 21 Cost : 0.549
Step : 31 Cost : 0.549
Step : 41 Cost : 0.549
Step : 51 Cost : 0.549
Step : 61 Cost : 0.549
Step : 71 Cost : 0.549
Step : 81 Cost : 0.549
Step : 91 Cost : 0.549


In [15]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-100'

In [16]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print('prediction: ', sess.run(prediction, feed_dict={X: x_data}))
print('target: ', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("accuracy : %.2f" % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

prediction:  [0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2]
target:  [0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2]
accuracy : 100.00


- `tf.argmax` : 배열에서 가장 큰 값의 인덱스를 반환하는 함수. np.argmax 도 동일
- `tf.cast` : 값은 다른 데이터 타입으로 캐스팅하는 함수